## Checkpoints
- [ ] ``load -> save -> compare``
- [ ] ``make pkg from scratch``
- [ ] ``layers generator``

In [10]:
upgrade_py = True
if upgrade_py:
    !python3.10 -m pip install --upgrade pip
    !pip install graph-notebook
    !pip install graphviz
    !pip install pydot
    !pip install networkx
    !pip install pyvis
    !pip install platform
    !pip install hashlib
    !pip install jycm
    !pip install pipreqs
%load_ext line_profiler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.0 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2


ERROR: Could not find a version that satisfies the requirement platform (from versions: none)
ERROR: No matching distribution found for platform
ERROR: Ignored the following yanked versions: 20081119
ERROR: Could not find a version that satisfies the requirement hashlib (from versions: none)
ERROR: No matching distribution found for hashlib
  Preparing metadata (setup.py) ... done
  Using cached nbconvert-7.16.4-py3-none-any.whl.metadata (8.5 kB)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 3.9 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=1a08d729dfef52002ebad35e0aa123f33dce1a723d9b46b2c31a2a1dcaf14f14
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
  Attempting uninstall: ipython
    Found existing installation: ipython 8.10.0
    Uninstalling ipython-8.10.0:
      Successfully uninstalled ipython-8.10.0
  Attempting uninstall: nbconvert
    Found existing installation: nbconvert 7.2.8
    Uninstalling nbconvert-7.2.8:
      Successfully uninstalled nbconvert-7.2.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
graph-notebook 4.5.2 requires ipython<=8.10.0,>=7.16.1, but you have ipython 8.12.3 which is incompatible.
graph-notebook 4.5.2 require

In [10]:
!pipreqs --print --scan-notebooks .

TypeError: isinstance() arg 2 must be a type, a tuple of types, or a union

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
import os
import sys
import matplotlib.pyplot as plt
import networkx as nx
import json
import pprint
import numpy as np
import time
import platform
import hashlib
from enum import Enum
from pyvis.network import Network
from IPython.display import display, HTML
import copy
from jycm.jycm import YouchamaJsonDiffer

from pathlib import Path


In [5]:
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(Path(os.getcwd()).parent.absolute())

from common import *

In [6]:
pkg = Package()
pkg.loadFile('data/namespacetest_rev0.pkg')
pkg.counts()


pkg ok
inputs:9, outputs:10, neurons:10, links:10, total:39


In [7]:
def showGraph(pkg):
    #net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
    nt = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook=True, directed =True)

    ZH = "ZERO_HOLE"
    def hasNode(nodes, key)->bool:
        for node in nx_graph.nodes:
            if str(node) == str(key):
                return True
        return False
    nx_graph = nx.DiGraph()

    for n in pkg.neurons:
        nx_graph.add_node(str(n["id"]), size = 10 + int(float(n["currentEnergy"])), title=('neuron %d' % n["id"]), group=1)
    for o in pkg.outputs:
        nx_graph.add_node(str(o["id"]), size = 10, title=('outputs %d' % n["id"]), group=2)
    for i in pkg.inputs:
        nx_graph.add_node(str(i["name"]), size = 5, title=('input %s' % i["name"]), group=3)

    #make direct links
    nx_graph.add_node(str(ZH), size = 1, title=("missed node"), group=1)
    l = pkg.links[1]
    start, stop = pkg.linkEnds(l)
    #print("link ", start, " to " , stop)
    #nx_graph.add_edge(str(start), str(stop))

    for l in pkg.links:
        print(l)
        start, stop = pkg.linkEnds(l)
        link_len = int(l['length'])
        print("link ", start, " to " , str(stop), " sz:", link_len)
        if not start:
            continue
            nx_graph.add_edge(ZH, stop, length = (link_len + 1), weight=(link_len + 1))
        if not hasNode(nx_graph.nodes, str(stop)):
            continue
            nx_graph.add_edge(ZH, stop, value = (link_len + 1), weight=(link_len + 1))
        nx_graph.add_edge(str(start), str(stop), length = (link_len + 1), weight=(link_len + 1) )

    for n in pkg.neurons:
        for rc in n['receivers']:
            if not pkg.isLink(rc):
                nx_graph.add_edge(str(n["id"]), str(rc), length = 1, weight = 1)
    nt.from_nx(nx_graph)
    return nt.show('nx.html')

In [8]:
showGraph(pkg)

{'attenuationPerTick': '0.020000', 'events': [], 'id': 10, 'length': 2, 'receiverId': 64}
link 10 is hanging down
link  0  to  64  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 11, 'length': 2, 'receiverId': 65}
link  Alias1  to  65  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 12, 'length': 2, 'receiverId': 66}
link  Alias2  to  66  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 13, 'length': 2, 'receiverId': 67}
link  Alias3  to  67  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 14, 'length': 2, 'receiverId': 68}
link  Alias4  to  68  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 15, 'length': 2, 'receiverId': 69}
link  Alias5  to  69  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 16, 'length': 2, 'receiverId': 70}
link  Alias6  to  70  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 17, 'length': 2, 'receiverId': 71}
link  Alias7  to  71  sz: 2
{'attenuationPerTick': '0.020000', 'e

In [9]:
## SAVELOAD TEST
saved_data = pkg.dump()
dump_fname = "data/TEST_out.json"
pkg.saveToFile(dump_fname)
#pkg.saveToFile()
saved_data

restored = pkg.loadFile(dump_fname)
showGraph(restored)

pkg ok


AttributeError: 'NoneType' object has no attribute 'neurons'

In [ ]:
p = pkg.clone()
n1 = ObjectFactory.makeNeuronData(peak = 10.0, receivers = [], leak = 0.1, currentEnergy = 3.0, id = p.nextId())
p.neurons.append(n1)
p.connect(n1["id"], 64)
p.connect(n1["id"], 65)
p.connect(n1["id"], 69)
g = showGraph(p)
print("new object",n1["id"])
g